# Replication Strategies: Master-Slave & Master-Master

Database replication is a technique for maintaining multiple copies of data across different database servers. This ensures **high availability**, **fault tolerance**, and **read scalability**.

---

## Table of Contents
1. [Synchronous vs Asynchronous Replication](#sync-vs-async)
2. [Master-Slave (Single Leader) Pattern](#master-slave)
3. [Master-Master (Multi-Leader) Pattern](#master-master)
4. [Python Simulation: Replication Lag](#simulation)
5. [Trade-offs: Consistency vs Availability vs Latency](#tradeoffs)
6. [Key Takeaways](#takeaways)

<a id='sync-vs-async'></a>
## 1. Synchronous vs Asynchronous Replication

The fundamental choice in replication is **when** the leader acknowledges a write to the client.

### Synchronous Replication

```
Client ──► Leader ──► Replica(s)
   │                      │
   │    Wait for ACK      │
   ◄──────────────────────┘
```

- **How it works**: The leader waits for **all** (or a quorum of) replicas to confirm the write before responding to the client.
- **Pros**:
  - Strong consistency guarantee
  - No data loss on leader failure
- **Cons**:
  - Higher latency (blocked on slowest replica)
  - Reduced availability if any replica is down

### Asynchronous Replication

```
Client ──► Leader ──────► Client ACK (immediate)
              │
              └──► Replica(s) (background)
```

- **How it works**: The leader acknowledges the write **immediately** after local persistence, replicating to followers in the background.
- **Pros**:
  - Low latency writes
  - High availability (replicas can lag)
- **Cons**:
  - Potential data loss if leader fails before replication
  - Replication lag can cause stale reads

### Semi-Synchronous Replication

A hybrid approach where the leader waits for **at least one** replica to acknowledge before responding:

| Approach | Consistency | Latency | Availability |
|----------|-------------|---------|-------------|
| Fully Synchronous | Strong | High | Low |
| Semi-Synchronous | Moderate | Medium | Medium |
| Asynchronous | Eventual | Low | High |

<a id='master-slave'></a>
## 2. Master-Slave (Single Leader) Replication

Also known as **Primary-Replica** or **Leader-Follower** replication.

### Architecture

```
                    ┌─────────────┐
     Writes ───────►│   MASTER    │
                    │  (Leader)   │
                    └──────┬──────┘
                           │
              Replication Stream (WAL/Binlog)
                           │
           ┌───────────────┼───────────────┐
           ▼               ▼               ▼
    ┌───────────┐   ┌───────────┐   ┌───────────┐
    │  SLAVE 1  │   │  SLAVE 2  │   │  SLAVE 3  │
    │ (Replica) │   │ (Replica) │   │ (Replica) │
    └───────────┘   └───────────┘   └───────────┘
           ▲               ▲               ▲
           └───────────────┴───────────────┘
                      Reads
```

### How It Works

1. **All writes** go to the master node
2. Master records changes in a **write-ahead log (WAL)** or **binary log**
3. Slaves continuously pull/receive changes and apply them
4. **Reads** can be served from master or any slave

### Use Cases

- **Read-heavy workloads**: Scale reads across multiple replicas
- **Analytics/Reporting**: Run heavy queries on slaves without impacting production
- **Geographic distribution**: Place read replicas closer to users
- **Disaster recovery**: Promote slave to master on failure

### Failover Process

```
1. Detect master failure (heartbeat timeout)
2. Choose most up-to-date slave
3. Promote slave to new master
4. Reconfigure other slaves to follow new master
5. Update application connection strings
```

### Challenges

| Challenge | Description |
|-----------|-------------|
| **Split Brain** | Network partition causes multiple masters |
| **Replication Lag** | Slaves may serve stale data |
| **Write Bottleneck** | Single master limits write throughput |
| **Failover Complexity** | Manual or automated promotion needed |

<a id='master-master'></a>
## 3. Master-Master (Multi-Leader) Replication

Also known as **Active-Active** or **Multi-Primary** replication.

### Architecture

```
     ┌─────────────┐         ┌─────────────┐
     │  MASTER A   │◄───────►│  MASTER B   │
     │ (Region US) │  Sync   │ (Region EU) │
     └──────┬──────┘         └──────┬──────┘
            │                       │
     Writes + Reads           Writes + Reads
            │                       │
      ┌─────┴─────┐           ┌─────┴─────┐
      │ US Users  │           │ EU Users  │
      └───────────┘           └───────────┘
```

### How It Works

1. **Multiple nodes** accept writes simultaneously
2. Each master replicates its changes to other masters
3. **Conflict resolution** required when same data modified on multiple masters

### Conflict Resolution Strategies

| Strategy | Description | Example |
|----------|-------------|----------|
| **Last Write Wins (LWW)** | Timestamp-based resolution | Cassandra, DynamoDB |
| **Application-Level** | Custom merge logic | CouchDB, Riak |
| **CRDTs** | Conflict-free data structures | Redis, Riak |
| **Operational Transform** | Transform concurrent ops | Google Docs |

### Use Cases

- **Multi-datacenter deployments**: Low-latency writes in each region
- **Offline-first applications**: Sync when connectivity restored
- **Collaborative editing**: Multiple users editing simultaneously

### Challenges

```
┌──────────────────────────────────────────────────────────┐
│                    CONFLICT SCENARIO                     │
├──────────────────────────────────────────────────────────┤
│  Time T1: User A (Master A) sets price = $100           │
│  Time T1: User B (Master B) sets price = $150           │
│                                                          │
│  After sync: Which value wins?                          │
│  - LWW: Use latest timestamp                            │
│  - Merge: Average? Higher? Custom logic?                │
└──────────────────────────────────────────────────────────┘
```

<a id='simulation'></a>
## 4. Python Simulation: Replication Lag

In [ ]:
import time
import random
import threading
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import statistics


@dataclass
class WriteOperation:
    """Represents a write operation with timestamp."""
    key: str
    value: any
    timestamp: float = field(default_factory=time.time)
    sequence: int = 0


class DatabaseNode:
    """Simulates a database node (master or replica)."""
    
    def __init__(self, name: str, is_master: bool = False):
        self.name = name
        self.is_master = is_master
        self.data: Dict[str, any] = {}
        self.write_log: List[WriteOperation] = []
        self.last_applied_sequence = 0
        self.replication_lag_history: List[float] = []
        self._lock = threading.Lock()
    
    def write(self, key: str, value: any) -> WriteOperation:
        """Perform a write operation (master only)."""
        if not self.is_master:
            raise Exception(f"{self.name} is a replica, cannot write!")
        
        with self._lock:
            op = WriteOperation(
                key=key,
                value=value,
                timestamp=time.time(),
                sequence=len(self.write_log) + 1
            )
            self.data[key] = value
            self.write_log.append(op)
            self.last_applied_sequence = op.sequence
            return op
    
    def read(self, key: str) -> Optional[any]:
        """Read a value from the node."""
        return self.data.get(key)
    
    def apply_replication(self, op: WriteOperation, network_delay: float = 0):
        """Apply a replicated write operation."""
        # Simulate network delay
        time.sleep(network_delay)
        
        with self._lock:
            self.data[op.key] = op.value
            self.write_log.append(op)
            self.last_applied_sequence = op.sequence
            
            # Calculate replication lag
            lag = time.time() - op.timestamp
            self.replication_lag_history.append(lag)


class MasterSlaveCluster:
    """Simulates a Master-Slave replication cluster."""
    
    def __init__(self, num_replicas: int = 3, replication_mode: str = "async"):
        self.master = DatabaseNode("Master", is_master=True)
        self.replicas = [
            DatabaseNode(f"Replica-{i+1}") 
            for i in range(num_replicas)
        ]
        self.replication_mode = replication_mode  # "sync" or "async"
        self.network_delays = [random.uniform(0.01, 0.1) for _ in range(num_replicas)]
    
    def write(self, key: str, value: any) -> WriteOperation:
        """Write to master and replicate to slaves."""
        op = self.master.write(key, value)
        
        if self.replication_mode == "sync":
            # Synchronous: wait for all replicas
            threads = []
            for replica, delay in zip(self.replicas, self.network_delays):
                t = threading.Thread(
                    target=replica.apply_replication, 
                    args=(op, delay)
                )
                threads.append(t)
                t.start()
            
            for t in threads:
                t.join()  # Wait for all replicas
        else:
            # Asynchronous: fire and forget
            for replica, delay in zip(self.replicas, self.network_delays):
                threading.Thread(
                    target=replica.apply_replication, 
                    args=(op, delay)
                ).start()
        
        return op
    
    def get_replication_stats(self) -> Dict:
        """Get replication lag statistics."""
        stats = {}
        for replica in self.replicas:
            if replica.replication_lag_history:
                stats[replica.name] = {
                    "avg_lag_ms": statistics.mean(replica.replication_lag_history) * 1000,
                    "max_lag_ms": max(replica.replication_lag_history) * 1000,
                    "min_lag_ms": min(replica.replication_lag_history) * 1000,
                    "applied_ops": replica.last_applied_sequence
                }
        return stats


print("✅ Replication simulation classes defined!")

In [ ]:
# Demonstrate Asynchronous Replication Lag

print("=" * 60)
print("ASYNCHRONOUS REPLICATION DEMO")
print("=" * 60)

async_cluster = MasterSlaveCluster(num_replicas=3, replication_mode="async")

# Perform multiple writes
for i in range(10):
    async_cluster.write(f"key_{i}", f"value_{i}")
    time.sleep(0.02)  # Small delay between writes

# Wait for async replication to complete
time.sleep(0.3)

# Check replication status
print("\n📊 Replication Statistics (Async Mode):")
print("-" * 40)

for name, stats in async_cluster.get_replication_stats().items():
    print(f"\n{name}:")
    print(f"  Average Lag: {stats['avg_lag_ms']:.2f} ms")
    print(f"  Max Lag:     {stats['max_lag_ms']:.2f} ms")
    print(f"  Min Lag:     {stats['min_lag_ms']:.2f} ms")
    print(f"  Applied Ops: {stats['applied_ops']}")

# Demonstrate stale read scenario
print("\n" + "=" * 60)
print("STALE READ DEMONSTRATION")
print("=" * 60)

# Write to master
async_cluster.write("latest_price", 100.00)

# Immediately read from replica (before replication completes)
master_value = async_cluster.master.read("latest_price")
replica_value = async_cluster.replicas[0].read("latest_price")  # May be stale!

print(f"\nMaster value:  {master_value}")
print(f"Replica value: {replica_value} (potentially stale!)")

In [ ]:
# Compare Synchronous vs Asynchronous Performance

def benchmark_replication(mode: str, num_writes: int = 50) -> Dict:
    """Benchmark write performance for a replication mode."""
    cluster = MasterSlaveCluster(num_replicas=3, replication_mode=mode)
    
    start_time = time.time()
    for i in range(num_writes):
        cluster.write(f"bench_key_{i}", i * 100)
    end_time = time.time()
    
    # Wait for async ops to complete
    if mode == "async":
        time.sleep(0.5)
    
    return {
        "mode": mode,
        "total_time_ms": (end_time - start_time) * 1000,
        "writes_per_sec": num_writes / (end_time - start_time),
        "avg_write_latency_ms": ((end_time - start_time) / num_writes) * 1000,
        "replication_stats": cluster.get_replication_stats()
    }


print("=" * 60)
print("SYNCHRONOUS vs ASYNCHRONOUS PERFORMANCE")
print("=" * 60)

# Run benchmarks
async_results = benchmark_replication("async")
sync_results = benchmark_replication("sync")

print("\n📈 Performance Comparison:")
print("-" * 50)
print(f"{'Metric':<25} {'Async':<15} {'Sync':<15}")
print("-" * 50)
print(f"{'Total Time (ms)':<25} {async_results['total_time_ms']:<15.2f} {sync_results['total_time_ms']:<15.2f}")
print(f"{'Writes/sec':<25} {async_results['writes_per_sec']:<15.2f} {sync_results['writes_per_sec']:<15.2f}")
print(f"{'Avg Latency (ms)':<25} {async_results['avg_write_latency_ms']:<15.2f} {sync_results['avg_write_latency_ms']:<15.2f}")

speedup = sync_results['total_time_ms'] / async_results['total_time_ms']
print(f"\n⚡ Async is {speedup:.1f}x faster than Sync for writes!")

In [ ]:
# Master-Master Conflict Simulation

@dataclass
class ConflictingWrite:
    """A write that may conflict with another."""
    node: str
    key: str
    value: any
    timestamp: float


class MasterMasterCluster:
    """Simulates a Multi-Master replication cluster with conflict resolution."""
    
    def __init__(self, resolution_strategy: str = "lww"):
        self.masters = {
            "US": DatabaseNode("Master-US", is_master=True),
            "EU": DatabaseNode("Master-EU", is_master=True)
        }
        self.resolution_strategy = resolution_strategy
        self.conflicts: List[Dict] = []
    
    def write(self, region: str, key: str, value: any):
        """Write to a specific master."""
        return self.masters[region].write(key, value)
    
    def simulate_concurrent_writes(self, key: str, us_value: any, eu_value: any):
        """Simulate concurrent writes to both masters."""
        # Both writes happen "simultaneously"
        us_write = ConflictingWrite(
            node="US", key=key, value=us_value, 
            timestamp=time.time()
        )
        eu_write = ConflictingWrite(
            node="EU", key=key, value=eu_value,
            timestamp=time.time() + random.uniform(-0.001, 0.001)  # Slight variance
        )
        
        # Apply locally
        self.masters["US"].data[key] = us_value
        self.masters["EU"].data[key] = eu_value
        
        # Detect and resolve conflict during sync
        conflict = self._resolve_conflict(us_write, eu_write)
        self.conflicts.append(conflict)
        return conflict
    
    def _resolve_conflict(self, write1: ConflictingWrite, write2: ConflictingWrite) -> Dict:
        """Resolve conflict using configured strategy."""
        if self.resolution_strategy == "lww":
            # Last Write Wins
            winner = write1 if write1.timestamp >= write2.timestamp else write2
            loser = write2 if winner == write1 else write1
            
            # Apply winning value to both
            for master in self.masters.values():
                master.data[winner.key] = winner.value
            
            return {
                "key": winner.key,
                "strategy": "Last Write Wins (LWW)",
                "winner": {"node": winner.node, "value": winner.value},
                "loser": {"node": loser.node, "value": loser.value},
                "timestamp_diff_ms": abs(write1.timestamp - write2.timestamp) * 1000
            }
        
        elif self.resolution_strategy == "merge":
            # Custom merge logic (example: sum numeric values)
            if isinstance(write1.value, (int, float)) and isinstance(write2.value, (int, float)):
                merged = write1.value + write2.value
            else:
                merged = f"{write1.value}|{write2.value}"  # Concatenate
            
            for master in self.masters.values():
                master.data[write1.key] = merged
            
            return {
                "key": write1.key,
                "strategy": "Merge",
                "merged_value": merged,
                "original_values": {"US": write1.value, "EU": write2.value}
            }


print("=" * 60)
print("MASTER-MASTER CONFLICT RESOLUTION")
print("=" * 60)

# Demonstrate Last Write Wins
lww_cluster = MasterMasterCluster(resolution_strategy="lww")

print("\n🔄 Scenario: Both regions update 'product_price' simultaneously")
print("-" * 50)

conflict = lww_cluster.simulate_concurrent_writes(
    key="product_price",
    us_value=99.99,
    eu_value=89.99
)

print(f"\nConflict detected on key: '{conflict['key']}'")
print(f"Resolution strategy: {conflict['strategy']}")
print(f"Winner: {conflict['winner']['node']} with value ${conflict['winner']['value']}")
print(f"Loser:  {conflict['loser']['node']} with value ${conflict['loser']['value']}")
print(f"Timestamp difference: {conflict['timestamp_diff_ms']:.4f} ms")

print("\n📦 Final state after resolution:")
for region, master in lww_cluster.masters.items():
    print(f"  {region}: product_price = ${master.data.get('product_price')}")

# Demonstrate Merge Strategy
print("\n" + "=" * 60)
print("MERGE STRATEGY EXAMPLE")
print("=" * 60)

merge_cluster = MasterMasterCluster(resolution_strategy="merge")

conflict = merge_cluster.simulate_concurrent_writes(
    key="inventory_count",
    us_value=50,  # US adds 50 items
    eu_value=30   # EU adds 30 items
)

print(f"\nConflict on key: '{conflict['key']}'")
print(f"Strategy: {conflict['strategy']}")
print(f"Original values: US={conflict['original_values']['US']}, EU={conflict['original_values']['EU']}")
print(f"Merged result: {conflict['merged_value']}")

<a id='tradeoffs'></a>
## 5. Trade-offs: Consistency vs Availability vs Latency

### The CAP Theorem Context

Replication strategies inherently involve trade-offs based on the **CAP theorem** (Consistency, Availability, Partition Tolerance).

```
                    CONSISTENCY
                        ▲
                       /│\
                      / │ \
           Sync      /  │  \      Sacrificed during
         Replication/   │   \     network partitions
                   /    │    \
                  /     │     \
                 ───────┼───────
                /       │       \
    AVAILABILITY◄───────┼────────►PARTITION
                        │         TOLERANCE
                        │
                   Async Replication
                   (eventual consistency)
```

### PACELC Extension

Beyond CAP, the **PACELC theorem** adds:
- **P**artition: Same as CAP
- **A**vailability vs **C**onsistency during partition
- **E**lse (no partition): **L**atency vs **C**onsistency trade-off

### Trade-off Matrix

| Strategy | Consistency | Availability | Write Latency | Read Latency | Failure Tolerance |
|----------|-------------|--------------|---------------|--------------|-------------------|
| **Master-Slave (Sync)** | Strong | Medium | High | Low | Limited |
| **Master-Slave (Async)** | Eventual | High | Low | Low | Good |
| **Master-Master (Async)** | Eventual | Very High | Low | Low | Excellent |
| **Master-Master (Sync)** | Strong | Low | Very High | Low | Excellent |

### When to Use Each Strategy

#### Master-Slave (Async) ✅ Best for:
- Read-heavy workloads (80%+ reads)
- Analytics and reporting
- When eventual consistency is acceptable
- Geographic read scaling

#### Master-Slave (Sync) ✅ Best for:
- Financial transactions requiring strong consistency
- Regulatory compliance (audit logs)
- When data loss is unacceptable

#### Master-Master ✅ Best for:
- Multi-region active-active deployments
- Low-latency writes in multiple geographies
- Offline-capable applications
- When conflict resolution is manageable

### Replication Lag Considerations

```
┌────────────────────────────────────────────────────────────┐
│              REPLICATION LAG IMPACT                        │
├────────────────────────────────────────────────────────────┤
│                                                            │
│  Read-After-Write Problem:                                 │
│  ┌─────────┐    Write    ┌─────────┐                      │
│  │  User   │ ──────────► │ Master  │                      │
│  └────┬────┘             └────┬────┘                      │
│       │                       │ (replicating...)          │
│       │    Read         ┌─────▼─────┐                     │
│       └───────────────► │  Replica  │ → Stale data!       │
│                         └───────────┘                     │
│                                                            │
│  Solutions:                                                │
│  • Read-your-writes: Route reads to master after write    │
│  • Monotonic reads: Stick to same replica                 │
│  • Bounded staleness: Max acceptable lag threshold        │
└────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize Trade-offs

trade_offs = {
    "Strategy": [
        "Master-Slave (Sync)",
        "Master-Slave (Async)",
        "Master-Master (Sync)",
        "Master-Master (Async)"
    ],
    "Consistency": [5, 2, 4, 2],      # 1-5 scale
    "Availability": [2, 4, 3, 5],
    "Write Latency (inverse)": [2, 5, 1, 4],  # Higher = lower latency
    "Complexity": [2, 2, 5, 4]
}

print("=" * 60)
print("REPLICATION STRATEGY COMPARISON (1-5 Scale)")
print("=" * 60)

header = f"{'Strategy':<25} {'Consistency':>12} {'Availability':>12} {'Low Latency':>12} {'Simplicity':>12}"
print(header)
print("-" * len(header))

for i, strategy in enumerate(trade_offs["Strategy"]):
    cons = "★" * trade_offs["Consistency"][i] + "☆" * (5 - trade_offs["Consistency"][i])
    avail = "★" * trade_offs["Availability"][i] + "☆" * (5 - trade_offs["Availability"][i])
    latency = "★" * trade_offs["Write Latency (inverse)"][i] + "☆" * (5 - trade_offs["Write Latency (inverse)"][i])
    simple = "★" * (6 - trade_offs["Complexity"][i]) + "☆" * (trade_offs["Complexity"][i] - 1)
    
    print(f"{strategy:<25} {cons:>12} {avail:>12} {latency:>12} {simple:>12}")

print("\n💡 Key: ★ = Better, ☆ = Worse")

<a id='takeaways'></a>
## 6. Key Takeaways

### 📌 Summary

| Concept | Key Point |
|---------|----------|
| **Synchronous Replication** | Strong consistency, higher latency, reduced availability during replica failures |
| **Asynchronous Replication** | Low latency, high availability, risk of data loss and stale reads |
| **Master-Slave** | Simple, good for read scaling, single point of write bottleneck |
| **Master-Master** | Multi-region writes, complex conflict resolution, excellent availability |

### 🎯 Decision Framework

```
                        Start
                          │
                          ▼
              ┌───────────────────────┐
              │ Need multi-region     │
              │ low-latency writes?   │
              └───────────┬───────────┘
                    │           │
                   YES          NO
                    │           │
                    ▼           ▼
          ┌─────────────┐  ┌─────────────┐
          │Master-Master│  │ Read-heavy  │
          │             │  │ workload?   │
          └──────┬──────┘  └──────┬──────┘
                 │               │     │
                 │              YES    NO
                 │               │     │
                 │               ▼     ▼
                 │    ┌────────────┐ ┌────────────┐
                 │    │Master-Slave│ │ Consider   │
                 │    │(Async)     │ │ Sync or    │
                 │    └────────────┘ │ Semi-Sync  │
                 │                   └────────────┘
                 ▼
    ┌────────────────────────────┐
    │ Can tolerate conflicts?    │
    └─────────────┬──────────────┘
            │           │
           YES          NO
            │           │
            ▼           ▼
      ┌──────────┐  ┌──────────┐
      │  Async   │  │   Sync   │
      │  (LWW)   │  │  (Heavy) │
      └──────────┘  └──────────┘
```

### 🔧 Best Practices

1. **Monitor replication lag** continuously in production
2. **Use read-your-writes consistency** for user-facing operations
3. **Design for eventual consistency** when possible to improve availability
4. **Test failover procedures** regularly
5. **Choose conflict resolution strategy** based on your data semantics
6. **Consider semi-synchronous** as a balanced middle ground

### 📚 Further Reading

- *Designing Data-Intensive Applications* by Martin Kleppmann (Chapters 5 & 9)
- CAP Theorem: Brewer's original paper
- PACELC: *Consistency Tradeoffs in Modern Distributed Database System Design*